# Project Title
### WORLD CITIES LAND TEMPERATURE ETL & DATA WAREHOUSE

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import logging
import boto3
from botocore.exceptions import ClientError
import psycopg2
from helpers.sql import sql_create_tables, staging_insert, sql_drop_tables
from configparser import ConfigParser

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [2]:
# Read in the data
directory = os.path.realpath("..") + '/global-land-temp-dw/data/'
tables = ['GlobalLandTemperaturesByMajorCity.csv', 'GlobalLandTemperaturesByCity.csv']

df_major_city = pd.read_csv(directory + tables[0])
df_city = pd.read_csv(directory + tables[1])

In [3]:
# Drop NA values This data requires all values to be available

def drop_na(pd_df):

    print(f'Number of rows before removing NA: {pd_df.shape[0]:,}')

    pd_df.dropna(axis=0, inplace=True)

    print(f'Number of rows after removing NA: {pd_df.shape[0]:,}')


drop_na(df_major_city)
drop_na(df_city)


Number of rows before removing NA: 239,177
Number of rows after removing NA: 228,175
Number of rows before removing NA: 8,599,212
Number of rows after removing NA: 8,235,082


In [4]:
# Add column for Boolean Major city
df_major_city = df_major_city.assign(major_city='true')
df_city = df_city.assign(major_city='false')



In [5]:
# Concatnate both datasets
dframes = [df_major_city, df_city]
df = pd.concat(dframes, ignore_index=True)

In [6]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,major_city
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W,true
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W,true
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W,true
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W,true
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W,true


In [7]:
# Sort data by year
df.sort_values(by='dt',inplace=True)

In [8]:
df.head(1000)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,major_city
8069490,1743-11-01,7.067,1.839,West Bromwich,United Kingdom,52.24N,2.63W,false
4466803,1743-11-01,2.840,1.887,Lvov,Ukraine,50.63N,24.08E,false
7872899,1743-11-01,7.807,1.816,Venice,Italy,45.81N,12.69E,false
7070831,1743-11-01,6.176,2.264,Stara Zagora,Bulgaria,42.59N,26.18E,false
4457264,1743-11-01,7.541,1.753,Luton,United Kingdom,52.24N,0.00W,false
...,...,...,...,...,...,...,...,...
4750557,1744-04-01,8.810,2.688,Mazyr,Belarus,52.24N,28.91E,false
2289987,1744-04-01,12.943,2.166,Edirne,Turkey,42.59N,26.18E,false
339969,1744-04-01,11.596,2.044,Aix En Provence,France,44.20N,4.47E,false
4457265,1744-04-01,8.296,2.501,Luton,United Kingdom,52.24N,0.00W,false


In [9]:
# Confirm Data types are as expected
df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
major_city                        object
dtype: object

In [ ]:
# Save data to CSV to load into SQL redshift
df.to_csv(directory + "dataset.csv")

In [ ]:
# Upload to s3 Function from AWS Services https://boto3.amazonaws.com/v1/documentation/api/1.10.46/guide/s3-uploading-files.html

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
# Get Bucket Name from configurations 
config = ConfigParser()
config.read('DW.cfg')
file_log = config.get('S3', 'log_data')

upload_file(directory + "dataset.csv", file_log, "logs/log_data.csv")

#s3 = boto3.client('s3')
#with open(directory + "dataset.csv", "rb") as f:
#    s3.upload_fileobj(f, "global-land-temp", "data_1")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Star Schema

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Get Database Variables
config = ConfigParser()
config.read('DW.cfg')

print("{}, {}, {}, {}, {}".format(*config['CLUSTER'].values()))


# Connect to redshift database
conn = psycopg2.connect("""host={} 
                           dbname={} 
                           user={} 
                           password={}
                           port={}"""\
                           .format(*config['CLUSTER'].values()    
))

# conn.set_session(autocommit=True)
cur = conn.cursor()


In [ ]:
# Drop all table if exists
for query in sql_drop_tables:
    cur.execute(query)
    conn.commit()

In [ ]:
# Create tables
for query in sql_create_tables:
    cur.execute(query)
    conn.commit()


In [ ]:
# Copy data to Staging
sql_query = staging_insert.format(
    's3://global-land-temp/logs/',
    config.get('IAM_ROLE','ARN'),
    config.get('CLUSTER','dwh_region')
    )
print(sql_query)
cur.execute(sql_query)
conn.commit()

In [ ]:
for query in sql_create_tables:
    print(query)


In [ ]:
# Write code here

In [ ]:
# Close Connection to the DB
conn.close()

In [ ]:
# Perform quality checks here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.